In [1]:
import random
import json
import pickle
import numpy as np
import tensorflow 
import keras
import nltk 
#nltk.download()

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Dropout
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.models import load_model

In [2]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [3]:
intents = json.loads(open('intents.json').read())

In [4]:
words = []
classes = []
documents = []
ignore_letters = ['?', '!', '.', ',']

for intent in intents['intents']:
    for pattern in intent['patterns']:
        word_list = nltk.word_tokenize(pattern)
        words.extend(word_list)
        documents.append((word_list, intent['tag']))
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

In [5]:
words = [word for word in words if word not in ignore_letters]
words = sorted(set(words))
classes = sorted(set(classes))

In [6]:
pickle.dump(words, open('words.pkl', 'wb'))
pickle.dump(classes, open('classes.pkl', 'wb'))

In [7]:
training = []
output_empty = [0] * len(classes)

for document in documents:
    bag = []
    word_patterns = document[0]
    for word in words:
        bag.append(1) if word in word_patterns else bag.append(0)

    output_row = list(output_empty)
    output_row[classes.index(document[1])] = 1
    training.append([bag, output_row])

In [8]:
random.shuffle(training)
training = np.array(training)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  


In [9]:
train_x = list(training[:, 0])
train_y = list(training[:, 1])

In [10]:
model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),),activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))

In [11]:
sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/gradient_descent.py:102: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(SGD, self).__init__(name, **kwargs)


In [12]:
hist = model.fit(np.array(train_x), np.array(train_y), epochs=200, batch_size=5, verbose=1)

Epoch 1/200
10/10 [==============================] - 1s 3ms/step - loss: 2.2691 - accuracy: 0.1064
Epoch 2/200
10/10 [==============================] - 0s 3ms/step - loss: 2.1656 - accuracy: 0.0638
Epoch 3/200
10/10 [==============================] - 0s 2ms/step - loss: 2.0933 - accuracy: 0.1915
Epoch 4/200
10/10 [==============================] - 0s 2ms/step - loss: 2.0470 - accuracy: 0.2979
Epoch 5/200
10/10 [==============================] - 0s 2ms/step - loss: 1.9028 - accuracy: 0.4043
Epoch 6/200
10/10 [==============================] - 0s 2ms/step - loss: 1.9031 - accuracy: 0.3191
Epoch 7/200
10/10 [==============================] - 0s 2ms/step - loss: 1.6904 - accuracy: 0.5319
Epoch 8/200
10/10 [==============================] - 0s 4ms/step - loss: 1.6221 - accuracy: 0.5106
Epoch 9/200
10/10 [==============================] - 0s 2ms/step - loss: 1.5114 - accuracy: 0.6170
Epoch 10/200
10/10 [==============================] - 0s 2ms/step - loss: 1.4307 - accuracy: 0.6170
Epoch 11/

In [13]:
model.save('chatbotmodel.h5', hist)

## **Chatbot**

In [14]:
intents = json.loads(open('intents.json').read())

words = pickle.load(open('words.pkl', 'rb'))
classes = pickle.load(open('classes.pkl', 'rb'))
model = load_model('chatbotmodel.h5')

In [15]:
def clean_up_sentence(sentence):
	sentence_words = nltk.word_tokenize(sentence)
	return sentence_words

In [16]:
def bag_of_words(sentence):
	sentence_words = clean_up_sentence(sentence)
	bag = [0] * len(words)
	for w in sentence_words:
		for i, word in enumerate(words):
			if word == w:
				bag[i] = 1
	return np.array(bag)

In [17]:
def predict_class(sentence):
	bow = bag_of_words(sentence)
	res = model.predict(np.array([bow]))[0]
	ERROR_THRESHOLD = 0.25
	results = [[i, r] for i, r in enumerate(res) if r > ERROR_THRESHOLD]

	results.sort(key=lambda x: x[1], reverse=True)
	return_list = []
	for r in results:
		return_list.append({'intent': classes[r[0]], 'probability': str(r[1])})
	return return_list

In [20]:
def get_response(intents_list, intents_json):
	tag = intents_list[0]['intent']
	list_of_intents = intents_json['intents']
	for i in list_of_intents:
		if i['tag'] == tag:
			result = random.choice(i['responses'])
			break
	return result

In [ ]:
while True:
	message = input("Enter a message:")
	ints = predict_class(message)
	res = get_response(ints, intents)
	print(res)

Enter a message:hi
Hello, thanks for asking
Enter a message:how are you
Good to see you again
Enter a message:
Hello, thanks for asking
Enter a message:q
Good to see you again
Enter a message:What you can do?
I can guide you through Adverse drug reaction list, Blood pressure tracking, Hospitals and Pharmacies
Enter a message:How to check Adverse drug reaction?
Navigating to Adverse drug reaction module
Enter a message:thnks alot
Good to see you again
Enter a message:thanks
Happy to help!
Enter a message:bye
See you!
